#***LINEAR*** ***REGRESSION***




In [ ]:
import pandas as pd
import numpy as np
import random
import time
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#Linear Algebra way for linear regression

In [ ]:
X,y = load_diabetes(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 46)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(r2_score(y_test,y_pred))
print(lr.intercept_)
print(lr.coef_)

0.4411325916951332
150.0663440989284
[ 1.35669119e+01 -2.10613526e+02  5.21704437e+02  3.35877974e+02
 -7.86143093e+02  4.45357851e+02  1.77438991e+02  2.57841824e+02
  7.89195708e+02 -1.05677993e-01]


In [ ]:
class LALR_Regression:

  def __init__(self):
    self.intercept = 0
    self.coef = 0

  def fit(self,X_train,y_train):
    X_train = np.insert(X_train,0,1,axis=1)

    result = np.linalg.inv( np.dot(X_train.T, X_train)).dot( np.dot(X_train.T, y_train) )
    self.intercept = result[0]
    self.coef = result[1:]

    print(self.intercept)
    print(self.coef)

  def predict(self,X_test):
    return np.dot(X_test, self.coef) + self.intercept


md = LALR_Regression()
md.fit(X_train, y_train)
y_pred = md.predict(X_test)
print(r2_score(y_test,y_pred))

150.06634409892834
[ 1.35669119e+01 -2.10613526e+02  5.21704437e+02  3.35877974e+02
 -7.86143093e+02  4.45357851e+02  1.77438991e+02  2.57841824e+02
  7.89195708e+02 -1.05677993e-01]
0.44113259169513375


# Gradient Descent

### Batch Gd

In [ ]:
X,y = load_diabetes(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 25)

In [ ]:
class BGD_Regressor:
  def __init__(self, lr= 0.01, epoch = 100):
    self.lr = lr
    self.epoch = epoch

  def fit(self, X_train, y_train):
    self.coef = np.ones([X_train.shape[1],])
    self.intercept = 0

    for i in range(self.epoch):
      y_pred = np.dot(X_train, self.coef) + self.intercept

      inter_der = -2 * np.mean(y_train - y_pred)

      derivative = -2 * np.dot((y_train - y_pred),X_train)/X_train.shape[0]

      self.coef -= self.lr * derivative
      self.intercept -= self.lr * inter_der

    print(self.intercept)
    print(self.coef, self.coef.shape)


  def predict(self,X_test):
    return np.dot(X_test, self.coef) + self.intercept


md = BGD_Regressor(.7, 300)
md.fit(X_train,y_train)
y_pred = md.predict(X_test)
print(r2_score(y_test,y_pred))


152.24164517800864
[  53.06170858  -72.11243384  373.01253576  304.61059172   -9.42092628
  -61.51657456 -191.03298912  137.21429911  325.96852732  115.07262459] (10,)
0.4335093033483943


In [ ]:
LinReg  = LinearRegression()
LinReg.fit(X_train,y_train)
y_pred = LinReg.predict(X_test)
print(LinReg.intercept_)
print(LinReg.coef_)
print(r2_score(y_test,y_pred))

151.96412304279053
[  -4.4110389  -176.14975887  515.70916826  415.68912805 -682.69131823
  328.41582506   92.97442489  210.1419437   700.10965675    4.96125582]
0.4373000617984171


###Stochastic GD

In [ ]:
X,y = load_diabetes(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 27)

In [ ]:
class SGD_Regressor:
  def __init__(self, lr = 0.01, epoch = 1000, adaptive_lr = False):
    self.lr = lr
    self.epoch = epoch
    self.adaptive_lr = adaptive_lr

  def fit(self, X_train, y_train):
    rows, col = X_train.shape
    self.intercept = 0
    self.coef = np.ones([col,])
    self.start_time = time.time()
    lr = self.lr
    if(self.adaptive_lr):
        print("Initial lr: ", lr)

    for i in range(self.epoch):
      for j in range(rows):

        id = np.random.randint(0,rows)
        y_pred = np.dot(X_train[id], self.coef) + self.intercept

        intercept_diff = -2 * (y_train[id] - y_pred)
        coef_diff = -2 * np.dot((y_train[id] - y_pred) ,X_train[id])

        if(intercept_diff < 1e-4 and coef_diff.all() < 1e-4):
          print("Reached convergence before the end of epoches")
          print(self.intercept)
          print(self.coef)
          return


        if(self.adaptive_lr):
          lr = self.schedule()

        self.intercept -= self.lr * intercept_diff
        self.coef -= self.lr * coef_diff


    print(self.intercept)
    print(self.coef)
    if(self.adaptive_lr):
        print("Final lr: ", lr)

  def schedule(self):
    t0 = self.lr
    t = time.time() - self.start_time
    return t0/(1+t)

  def predict(self, X_test):
    return np.dot(X_test, self.coef) + self.intercept


md = SGD_Regressor(lr = .01,epoch = 500, adaptive_lr = True)
md.fit(X_train,y_train)
y_pred = md.predict(X_test)
print(r2_score(y_test,y_pred))

Initial lr:  0.01
156.22906558488546
[  -6.57670206 -300.52455456  485.0513865   399.30240987  -82.68580897
  -77.45643725 -214.81564986  112.06207057  480.29335001   79.85202978]
Final lr:  0.0018623691712769247
0.4032520292013083


In [ ]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor()
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
print(sgd.intercept_)
print(sgd.coef_)
print(r2_score(y_test,y_pred))

[151.34576838]
[  56.74444007  -35.99795472  273.88284985  172.56738214   35.34318523
   10.97722499 -155.29066028  143.49524708  232.39871962  122.83770802]
0.3959456432008617


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


###Mini Batch GD

In [ ]:
X,y = load_diabetes(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 36)

In [ ]:
class MBGD_Regressor():
  def __init__(self, batch_size,lr = 0.01, epoch =100, adaptive_lr = False):
    self.lr = lr
    self.epoch = epoch
    self.batch_size = batch_size
    self.adaptive_lr = adaptive_lr

  def fit(self, X_train, y_train):
    self.intercept = 0
    self.coef = np.ones(X_train.shape[1])
    batches = int(X_train.shape[0]/self.batch_size)
    self.start_time = time.time()
    lr = self.lr

    if(self.adaptive_lr):
      print("Initial lr: ",lr)

    for i in range(self.epoch):
      for j in range(batches):

        id = random.sample(range(X_train.shape[0]), self.batch_size)

        y_pred = np.dot(X_train[id], self.coef) + self.intercept

        intercept_diff = -2 * np.mean(y_train[id] - y_pred)
        coef_diff = -2 * np.dot((y_train[id] - y_pred), X_train[id])/self.batch_size

        if(self.adaptive_lr):
          lr = self.schedule()

        self.intercept -= lr * intercept_diff
        self.coef -= lr * coef_diff

    print(self.intercept)
    print(self.coef)
    if(self.adaptive_lr):
      print("Initial lr: ",lr)

  def schedule(self):
    t0 = self.lr
    t = time.time() - self.start_time
    return t0/(1+t)

  def predict(self,X_test):
    return np.dot(X_test, self.coef) + self.intercept

md = MBGD_Regressor(lr = 0.09, epoch = 300, batch_size = 30, adaptive_lr = True)
md.fit(X_train,y_train)
y_pred = md.predict(X_test)
r2_score(y_test,y_pred)

Initial lr:  0.09
152.3524330088306
[  29.04197132 -127.16547415  382.55789841  286.73672547    6.2471835
  -38.48199936 -183.00169582  136.90618076  323.86096984  144.61520305]
Initial lr:  0.050643381647619196


0.41526244647623145

In [ ]:
class sklearn_MGBD:
  def __init__(self, batch_size,lr = 0.01, epoch =100):
    self.epoch = epoch
    self.batch_size = batch_size
    self.SGD = SGDRegressor(learning_rate = 'constant', eta0 = lr)

  def fit(self,X_train,y_train):
    batches = int(X_train.shape[0]/self.batch_size)

    for i in range(self.epoch):
      for j in range(batches):

        id = random.sample(range(X_train.shape[0]), self.batch_size)
        self.SGD.fit(X_train[id], y_train[id])

    print(self.SGD.intercept_)
    print(self.SGD.coef_)

  def predict(self, X_test):
    return self.SGD.predict(X_test)

MGBD = sklearn_MGBD(lr = 0.007, epoch = 200, batch_size = 50)
MGBD.fit(X_train,y_train)
y_pred = MGBD.predict(X_test)
r2_score(y_test, y_pred)